In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import torch
import torch.nn as nn

In [ ]:
#defining the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#FMNIST Classses
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
#installing FMNIST dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full.shape

In [ ]:
#normalizing the data
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

In [ ]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

#convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test = keras.utils.to_categorical(y_test, 10)
y_train_full = keras.utils.to_categorical(y_train_full, 10)

In [ ]:
def convert_array_to_rgb(images):
    # Create an empty list to store converted RGB images
    rgb_images = []
    
    # Iterate over each image in the array
    for image in images:
        # Duplicate the single channel to create three channels
        rgb_image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
        rgb_images.append(rgb_image)
    
    # Convert the list of RGB images to a NumPy array
    rgb_images = np.array(rgb_images)
    
    return rgb_images

In [ ]:
X_train_rgb = convert_array_to_rgb(X_train)
X_valid_rgb = convert_array_to_rgb(X_valid)
X_test_rgb = convert_array_to_rgb(X_test)
X_train_full_rgb = convert_array_to_rgb(X_train_full)

X_train_rgb.shape

In [ ]:
import torch
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batch_norm1 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batch_norm2 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batch_norm3 = nn.BatchNorm2d(256)

        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=1)  # default stride is 2
        self.batch_norm4 = nn.BatchNorm2d(512)

        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=1)  # default stride is 2
        self.batch_norm5 = nn.BatchNorm2d(512)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, 4096)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 4096)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.batch_norm1(x)

        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.batch_norm2(x)

        x = torch.relu(self.conv5(x))
        x = torch.relu(self.conv6(x))
        x = torch.relu(self.conv7(x))
        x = self.pool3(x)
        x = self.batch_norm3(x)

        x = torch.relu(self.conv8(x))
        x = torch.relu(self.conv9(x))
        x = torch.relu(self.conv10(x))
        x = self.pool4(x)
        x = self.batch_norm4(x)

        x = torch.relu(self.conv11(x))
        x = torch.relu(self.conv12(x))
        x = torch.relu(self.conv13(x))
        x = self.pool5(x)
        x = self.batch_norm5(x)

        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        features = torch.clone(x.cpu().detach())
        x = self.fc3(x)
        x = F.softmax(x, dim = 1)
        return x, features

# Create an instance of the model
model = CustomCNN()
print(model)
# model.to(device)

In [ ]:
model = CustomCNN()
model.load_state_dict(torch.load('/kaggle/input/keytiti/VGG16.pt', map_location=torch.device('cpu')))
model.eval()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

loss_object = nn.CrossEntropyLoss()

def create_adversarial_pattern(input_image, input_label, model):
    # Create a new tensor with requires_grad enabled
    input_image = input_image.clone().detach().requires_grad_(True)
    
    # Forward pass
    prediction = model(input_image)
    
    # If the model returns a tuple, extract the logits
    if isinstance(prediction, tuple):
        prediction = prediction[0]  # Assuming the logits are the first element
    
    # Calculate loss
    loss = loss_object(prediction, input_label.argmax(dim=1))
    
    # Backpropagation to compute gradients
    model.zero_grad()
    loss.backward()
    
    # Get the gradient of the input image
    gradient = input_image.grad.data
    
    # Convert the gradient to signed gradients
    signed_grad = torch.sign(gradient)
    
    return signed_grad


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move input tensors to the same device as the model
image = image.to(device)
image_label = image_label.to(device)

# Perturbations
perturbations = create_adversarial_pattern(image, image_label, model)

# Print original class prediction
image_pred = model(image)

print("Original class prediction:", image_pred)


In [ ]:
import matplotlib.pyplot as plt

# Move perturbations to CPU before visualization
perturbations_cpu = perturbations.cpu()

# Show the perturbations
plt.imshow(perturbations_cpu[0].permute(1, 2, 0).numpy() * 0.5 + 0.5)
plt.show()


In [ ]:
#plotting the adversarial image
def display_images(image, description):
    plt.figure()
    plt.axis('off')
    plt.imshow(image)
    plt.title(description)
    plt.show()
    
def adversarial_pattern(image, label):
    perturbations = create_adversarial_pattern(image, label, model)
    return perturbations


In [ ]:
perturbations = adversarial_pattern(image, image_label)
adv_x = image + perturbations * 0.1

# Assuming adversarial_pattern, display_images functions are defined elsewhere

# Ensure adv_x is on CPU and convert it to a numpy array
adv_x_cpu = adv_x.cpu().detach().numpy()

# Clip the values between 0 and 1
adv_x_cpu = np.clip(adv_x_cpu, 0, 1)


adv_image = adv_x_cpu[0, 0]  

plt.figure()
plt.axis('off')
plt.imshow(adv_image, cmap='gray')  # Assuming grayscale image, so specifying cmap='gray'
plt.title('Adversarial Image')
plt.show()




In [ ]:
#predicting the adversarial image
adv_pred = model(adv_x)
adv_pred

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

# Define the device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to generate adversarial examples for a dataset
def generate_adversarial_dataset(X, y, model, epsilon=0.1):
    # Set the model to evaluation mode
    model.eval()
    
    adversarial_dataset = []
    
    for i in tqdm(range(len(X))):
        # Convert image and label to PyTorch tensors
        image = torch.tensor(X[i], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
        image_label = torch.tensor([y[i]]).to(device)  # Move label to the device

        # Generate perturbations
        perturbations = adversarial_pattern(image, image_label)
        
        # Create adversarial example
        adv_x = image + perturbations * epsilon
        
        # Ensure adv_x is on CPU and convert it to a numpy array
        adv_x_cpu = adv_x.cpu().detach().numpy()
        
        # Clip the values between 0 and 1
        adv_x_cpu = np.clip(adv_x_cpu, 0, 1)
        
        # Store adversarial example and label
        adversarial_dataset.append((adv_x_cpu.squeeze(0), y[i]))  # Remove batch dimension before storing
        
    return adversarial_dataset

# Generate adversarial dataset for training set
adversarial_train_set = generate_adversarial_dataset(X_train_full, y_train_full, model)

# Generate adversarial dataset for test set
adversarial_test_set = generate_adversarial_dataset(X_test, y_test, model)